In [3]:
pip install geopy

     |████████████████████████████████| 125 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 2.3 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install geopandas

     |████████████████████████████████| 1.1 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 18.5 MB 4.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

# Sample DataFrame with latitude and longitude columns
data = {
    'latitude': [34.0522, 34.0525, 34.0519, 34.0532, 34.0530, 34.0535, 50.8240566],
    'longitude': [-118.2437, -118.2434, -118.2431, -118.2440, -118.2425, -118.2445, 4.3742187],
}

df = pd.DataFrame(data)

# Function to create a distance matrix
def create_distance_matrix(coords):
    distance_matrix = []
    for i in range(len(coords)):
        row = []
        for j in range(len(coords)):
            coords_i = (coords['latitude'].iloc[i], coords['longitude'].iloc[i])
            coords_j = (coords['latitude'].iloc[j], coords['longitude'].iloc[j])
            distance = great_circle(coords_i, coords_j).meters
            print(f'coords_i = {coords_i} distance = {distance}')
            row.append(distance)
        distance_matrix.append(row)
    return distance_matrix

# Create a distance matrix
distance_matrix = create_distance_matrix(df)

# Apply DBSCAN clustering
eps = 500  # Epsilon parameter: maximum distance between two samples for one to be considered as in the neighborhood of the other
min_samples = 2  # Minimum number of samples in a neighborhood for a point to be considered as a core point
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
clusters = dbscan.fit_predict(distance_matrix)

# Add the cluster labels to the DataFrame
df['cluster'] = clusters

print(df)

coords_i = (34.0522, -118.2437) distance = 0.0
coords_i = (34.0522, -118.2437) distance = 43.32058643645249
coords_i = (34.0522, -118.2437) distance = 64.56268944273597
coords_i = (34.0522, -118.2437) distance = 114.57845499155918
coords_i = (34.0522, -118.2437) distance = 141.89863265036692
coords_i = (34.0522, -118.2437) distance = 162.2582250971223
coords_i = (34.0522, -118.2437) distance = 9035684.971983694
coords_i = (34.0525, -118.2434) distance = 43.320586436480426
coords_i = (34.0525, -118.2434) distance = 0.0
coords_i = (34.0525, -118.2434) distance = 72.21529966920973
coords_i = (34.0525, -118.2434) distance = 95.467386361827
coords_i = (34.0525, -118.2434) distance = 99.82965434589771
coords_i = (34.0525, -118.2434) distance = 150.44653011176842
coords_i = (34.0525, -118.2434) distance = 9035641.98104991
coords_i = (34.0519, -118.2431) distance = 64.56268944281766
coords_i = (34.0519, -118.2431) distance = 72.21529966933375
coords_i = (34.0519, -118.2431) distance = 0.0
coor

In [12]:
df = pd.read_csv('ar41_for_ulb_cleaned.csv')

In [13]:
import pandas as pd
import geopandas as gpd
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

# Assume df is your large DataFrame with latitude and longitude columns

# Function to create a distance matrix for a chunk of data
def create_distance_matrix_chunk(coords_chunk, coords_full):
    distance_matrix = []
    for i in range(len(coords_chunk)):
        row = []
        for j in range(len(coords_full)):
            coords_i = (coords_chunk['lat'].iloc[i], coords_chunk['lon'].iloc[i])
            coords_j = (coords_full['lat'].iloc[j], coords_full['lon'].iloc[j])
            distance = great_circle(coords_i, coords_j).meters
            row.append(distance)
        distance_matrix.append(row)
    return distance_matrix

# Function to apply DBSCAN clustering and compute the mean for a chunk of data
def dbscan_and_mean_chunk(coords_chunk, eps=500, min_samples=2):
    distance_matrix = create_distance_matrix_chunk(coords_chunk, df)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
    clusters = dbscan.fit_predict(distance_matrix)
    
    # Add cluster labels to the chunk
    coords_chunk['cluster'] = clusters
    
    # Compute the mean coordinates for each cluster
    cluster_means = coords_chunk.groupby('cluster').agg({
        'lat': 'mean',
        'lon': 'mean',
        'cluster': 'count'  # Optional: count of points in each cluster
    }).reset_index()
    
    return cluster_means

# Define the chunk size
chunk_size = 1000000

# Split the data into chunks
chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Apply DBSCAN and compute mean for each chunk in parallel
cluster_mean_results = [dbscan_and_mean_chunk(chunk) for chunk in chunks]

# Concatenate the results
final_cluster_means = pd.concat(cluster_mean_results)

print(final_cluster_means)

KeyboardInterrupt: 